In [ ]:
import numpy as np

from vpython import *

class GameState():
    def __init__(self):
        self.scene = display(title='Examples of Tetrahedrons', x=0, y=0, width=600, height=600, center=vector(0,0,0), background=vector(0,0,0))

        Nobstale = 20  # change this to have more or fewer atoms
        # Typical values
        L = 1 # container is a cube L on a side
        gray = color.gray(0.7) # color of edges of container
        mass = 4E-3/6E23 # helium mass
        self.r_car = 0.027 # wildly exaggerated size of helium atom
        self.r_obstale = 0.1
        self.r_sensor =  0.0035
        self.dt = 0.05
        #the limitation of space
        d = L/2 + self.r_obstale
        r = 0.005
        boxbottom = curve(color=gray, radius=r)
        boxbottom.append([vector(-d,-d,-d), vector(-d,-d,d), vector(d,-d,d), vector(d,-d,-d), vector(-d,-d,-d)])
        boxtop = curve(color=gray, radius=r)
        boxtop.append([vector(-d,d,-d), vector(-d,d,d), vector(d,d,d), vector(d,d,-d), vector(-d,d,-d)])
        vert1 = curve(color=gray, radius=r)
        vert2 = curve(color=gray, radius=r)
        vert3 = curve(color=gray, radius=r)
        vert4 = curve(color=gray, radius=r)
        vert1.append([vector(-d,-d,-d), vector(-d,d,-d)])
        vert2.append([vector(-d,-d,d), vector(-d,d,d)])
        vert3.append([vector(d,-d,d), vector(d,d,d)])
        vert4.append([vector(d,-d,-d), vector(d,d,-d)])
        #define the car and the obstales
        self.car = sphere (pos=vector(0.05,0.02,0.04), color = color.green, radius = self.r_car, make_trail=True, retain=20)
        self.obstales = []
        self.sensor_arms = []
        self.opos = []
        
        for i in range(Nobstale):
            x =L*random()-L/2
            y = L*random()-L/2
            z = L*random()-L/2
            self.obstales.append(sphere(pos=vector(x,y,z), radius=self.r_obstale, color=gray))
            self.opos.append(vector(x,y,z))
        self.car.v = vector (-0.02, -0.055, +0.075)
        #side = side - thk*0.5 - car.radius
        self.side = d - r*0.5 - self.car.radius
        self.show_sensor = True
        self.initialed =  False
        self.crashed = False
    def frame_step(self, action):
        if action == 0:
            self.car.v.x += 0.05
        elif action == 1:
            self.car.v.y += 0.05
        elif action == 2:
            self.car.v.z += 0.05
        elif action == 3:
            self.car.v.y -= 0.05
        elif action == 4:
            self.car.v.z -= 0.05
        elif action == 5:
            self.car.v.y -= 0.05
        while True:
            rate(200)

            self.car.pos = self.car.pos + self.car.v*self.dt
            if not self.initialed:
                for i in range(0,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 0)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(0,1,0), angle=i*pi/4)

                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 0)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(0,0,1), angle=i*pi/4)

                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 0)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(0,1,1), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 0)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(0,1,-1), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 1)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(1,0,1), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 1)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(-1,0,1), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 2)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(1,1,0), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)
                for i in range(1,8):
                    sonar_arm = self.make_sonar_arm(self.car.pos,self.r_car, 2)
                    for sensor in sonar_arm:
                        sen = sensor.rotate(origin=self.car.pos, axis=vector(-1,1,0), angle=i*pi/4)
                    self.sensor_arms.append(sonar_arm)

                self.initialed = True 
            else:
                self.update_sonar_arm(self.sensor_arms, self.car.v)

                
            readings = self.get_sonar_readings(self.sensor_arms, self.obstales)
            #print(len(readings))
            #print(readings)
            
            state = np.array([readings])   

            if self.car_is_crashed(readings):
                self.carshed = True
                reward = -500
                self.recover_from_crash(self.car.v)
            else:
                reward = -5 + int(self.sum_readins(readings) / 10)
                
            if not (self.side > self.car.pos.x > -self.side):
                self.car.v.x = - self.car.v.x
            elif not (self.side > self.car.pos.y > -self.side):
                self.car.v.y = - self.car.v.y
            elif not (self.side > self.car.pos.z > -self.side):
                self.car.v.z = - self.car.v.z   
           
           

    def sum_readins(self,readings):
        tot = 0
        for i in readings:
            tot += i
            return tot
    def recover_from_crash(self,v):
        while self.crashed:
            self.car.v = -20 * v
            
            self.crashed = False
            
 
            
    def car_is_crashed(self, readings):
        for i in readings:
            if i == 0:
                return True
        return False
            
    def check_collision(self, sensor, obstales):
        for obstale in obstales:
            if mag2(obstale.pos-sensor.pos) < (self.r_sensor + self.r_obstale)**2:
                return True
        return False
    def check_edge(self, sensor):
        if not (self.side > sensor.pos.x > -self.side):
            return True
        elif not (self.side > sensor.pos.y > -self.side):
            return True
        elif not (self.side > sensor.pos.z > -self.side):
            return True
        else:
            return False
    
    def get_sonar_readings(self, sensor_arms, obstales):
        readings = []
        
        for sensor_arm in sensor_arms:
            appened = False
            i = 0
            for sensor in sensor_arm:
                
                if self.check_collision(sensor, obstales) or self.check_edge(sensor):
                    readings.append(i)
                    appened = True
                    break
                i += 1
            if not appened:
                readings.append(i)
        return readings
    
    
    def update_sonar_arm(self,sensor_arms, v):
        interval = 0.005  # Default spread.
        arm_points = []
        
        
        # Make an arm. We build it flat because we'll rotate it about the
        # center later.
       
        for sensor_arm in sensor_arms:
            for sensor in sensor_arm:
                sensor.pos = sensor.pos + v*self.dt
                if not (self.side > sensor.pos.x > -self.side and self.side > sensor.pos.y > -self.side and self.side > sensor.pos.z > -self.side ):
                    sensor.visible = False
                else:
                    sensor.visible = True



       
    def make_sonar_arm(self, pos, r_car, condition):
        
        interval = 0.05  # Default spread.
        arm_points = []
        
        sensor_arm = []
        # Make an arm. We build it flat because we'll rotate it about the
        # center later.
        if condition == 0:
            
            for i in range(1, 10):
                arm_points.append(vector(pos.x + (interval * i), pos.y, pos.z))
        elif condition == 1:
            for i in range(1, 10):
                arm_points.append(vector(pos.x, pos.y + (interval * i), pos.z))
        elif condition == 2:
            for i in range(1, 10):
                arm_points.append(vector(pos.x, pos.y, pos.z + (interval * i)))
        for point in arm_points:
            sensor_arm.append(sphere(pos=point, radius=self.r_sensor, color=color.red))
            
        return sensor_arm
    

            
            
game_state = GameState()
while True:
    game_state.frame_step((np.random.randint(0,6)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>